In [90]:
import sympy
from sympy import Symbol

from mlscorecheck.core import (score_function_set, sqrt)
from mlscorecheck.core import fowlkes_mallows_index, markedness

In [91]:
tp = Symbol('tp')
fp = Symbol('fp')
tn = Symbol('tn')
fn = Symbol('fn')
eps = Symbol('eps')
p = Symbol('p')
n = Symbol('n')
beta = Symbol('beta')
fm = Symbol('fm')
mk = Symbol('mk')

In [92]:
tp_ = 20
tn_ = 30
p_ = 50
n_ = 100
fn_ = p_ - tp_
fp_ = n_ - tn_
fm_ = fowlkes_mallows_index(tp=tp_, fp=fp_, p=p_)
mk_ = markedness(tp=tp_, tn=tn_, fp=fp_, fn=fn_)

In [93]:
eq0 = fm - fowlkes_mallows_index(tp=tp, fp=fp, p=p)
eq1 = mk - markedness(tp=tp, tn=tn, fp=fp, fn=fn)
eq2 = p - tp - fn
eq3 = n - tn - fp

In [120]:
eq0, eq1, eq2

(fm - sqrt(tp**2/(p*(fp + tp))),
 mk - tn/(fn + tn) - tp/(fp + tp) + 1,
 -fn + p - tp)

In [94]:
# solving the three equations containing fp for fp
fp0 = sympy.solve(eq0, fp, dict=True)
fp1 = sympy.solve(eq1, fp, dict=True)
fp3 = sympy.solve(eq3, fp, dict=True)

In [95]:
fp0, fp1, fp3

[{fp: -tp + tp**2/(fm**2*p)}]
[{fp: tp*(-fn*mk - mk*tn + tn)/(fn*mk + fn + mk*tn)}]
[{fp: n - tn}]


In [96]:
# here the fp is eliminated and we have two equations
eq10 = fp0[0][fp] - fp3[0][fp]
eq11 = fp1[0][fp] - fp3[0][fp]

In [97]:
eq10, eq11

(-n + tn - tp + tp**2/(fm**2*p),
 -n + tn + tp*(-fn*mk - mk*tn + tn)/(fn*mk + fn + mk*tn))

In [98]:
# solving the three equations containing tp for tp
tp0 = sympy.solve(eq10, tp, dict=True)
tp1 = sympy.solve(eq11, tp, dict=True)
tp2 = sympy.solve(eq2, tp, dict=True)

In [99]:
tp0, tp1, tp2

([{tp: fm*(fm*p - sqrt(p*(fm**2*p + 4*n - 4*tn)))/2},
  {tp: fm*(fm*p + sqrt(p*(fm**2*p + 4*n - 4*tn)))/2}],
 [{tp: (-fn*mk*n + fn*mk*tn - fn*n + fn*tn - mk*n*tn + mk*tn**2)/(fn*mk + mk*tn - tn)}],
 [{tp: -fn + p}])

In [ ]:
# FORKING on picking one particular solution for tp in tp[0]

In [100]:
# here the tp is eliminated
eq20 = tp0[0][tp] - tp2[0][tp]
eq21 = tp1[0][tp] - tp2[0][tp]

In [101]:
eq20, eq21

(fm*(fm*p - sqrt(p*(fm**2*p + 4*n - 4*tn)))/2 + fn - p,
 fn - p + (-fn*mk*n + fn*mk*tn - fn*n + fn*tn - mk*n*tn + mk*tn**2)/(fn*mk + mk*tn - tn))

In [102]:
# solving for tn
tn0 = sympy.solve(eq20, tn, dict=True)
tn1 = sympy.solve(eq21, tn, dict=True)

In [119]:
tn0, tn1

([{tn: fm**2*p/4 + n - (fm**2*p + 2*fn - 2*p)**2/(4*fm**2*p)}],
 [{tn: (-2*fn*mk + mk*n + mk*p - p - sqrt(4*fn*mk*n + 4*fn*mk*p + mk**2*n**2 + 2*mk**2*n*p + mk**2*p**2 - 2*mk*n*p - 2*mk*p**2 + p**2))/(2*mk)},
  {tn: (-2*fn*mk + mk*n + mk*p - p + sqrt(4*fn*mk*n + 4*fn*mk*p + mk**2*n**2 + 2*mk**2*n*p + mk**2*p**2 - 2*mk*n*p - 2*mk*p**2 + p**2))/(2*mk)}])

In [ ]:
# selecting the tn1[0] solution

In [105]:
tn1_sol_0 = tn1[0][tn]

In [106]:
# tn1 case 0

In [107]:
eq30 = tn0[0][tn] - tn1[0][tn]

In [108]:
eq30

fm**2*p/4 + n - (-2*fn*mk + mk*n + mk*p - p - sqrt(4*fn*mk*n + 4*fn*mk*p + mk**2*n**2 + 2*mk**2*n*p + mk**2*p**2 - 2*mk*n*p - 2*mk*p**2 + p**2))/(2*mk) - (fm**2*p + 2*fn - 2*p)**2/(4*fm**2*p)

In [22]:
# supposing that mk > 0 and fm > 0

In [109]:
eq30 = sympy.simplify(eq30 * (2*mk*4*fm**2*p))

In [110]:
print(eq30)

4*fm**2*mk*n*p + 4*fm**2*mk*p**2 + 4*fm**2*p**2 + 4*fm**2*p*sqrt(4*fn*mk*n + 4*fn*mk*p + mk**2*n**2 + 2*mk**2*n*p + mk**2*p**2 - 2*mk*n*p - 2*mk*p**2 + p**2) - 8*fn**2*mk + 16*fn*mk*p - 8*mk*p**2


In [111]:
expr = 4*fm**2*p*sqrt(4*fn*mk*n + 4*fn*mk*p + mk**2*n**2 + 2*mk**2*n*p + mk**2*p**2 - 2*mk*n*p - 2*mk*p**2 + p**2)

In [112]:
eq30_mod = sympy.simplify(sympy.Eq((eq30 - expr)**2, (-expr)**2))

In [113]:
eq30_mod = eq30_mod.lhs - eq30_mod.rhs

In [114]:
eq30_mod

-16*fm**4*p**2*(4*fn*mk*n + 4*fn*mk*p + mk**2*n**2 + 2*mk**2*n*p + mk**2*p**2 - 2*mk*n*p - 2*mk*p**2 + p**2) + 16*(fm**2*mk*n*p + fm**2*mk*p**2 + fm**2*p**2 - 2*fn**2*mk + 4*fn*mk*p - 2*mk*p**2)**2

In [115]:
fn_sol_0 = sympy.solve(eq30_mod, fn, dict=True)

In [116]:
fn_sol_0

[{fn: p},
 {fn: p - (9*p**2 - 3*(-fm**2*mk*n*p - fm**2*mk*p**2 - fm**2*p**2 + 3*mk*p**2)/mk)/(3*(-27*p**3 + sqrt(-4*(9*p**2 - 3*(-fm**2*mk*n*p - fm**2*mk*p**2 - fm**2*p**2 + 3*mk*p**2)/mk)**3 + (-54*p**3 + 27*p*(-fm**2*mk*n*p - fm**2*mk*p**2 - fm**2*p**2 + 3*mk*p**2)/mk + 27*(-fm**4*n*p**2 - fm**4*p**3 + fm**2*mk*n*p**2 + fm**2*mk*p**3 + fm**2*p**3 - mk*p**3)/mk)**2)/2 + 27*p*(-fm**2*mk*n*p - fm**2*mk*p**2 - fm**2*p**2 + 3*mk*p**2)/(2*mk) + 27*(-fm**4*n*p**2 - fm**4*p**3 + fm**2*mk*n*p**2 + fm**2*mk*p**3 + fm**2*p**3 - mk*p**3)/(2*mk))**(1/3)) - (-27*p**3 + sqrt(-4*(9*p**2 - 3*(-fm**2*mk*n*p - fm**2*mk*p**2 - fm**2*p**2 + 3*mk*p**2)/mk)**3 + (-54*p**3 + 27*p*(-fm**2*mk*n*p - fm**2*mk*p**2 - fm**2*p**2 + 3*mk*p**2)/mk + 27*(-fm**4*n*p**2 - fm**4*p**3 + fm**2*mk*n*p**2 + fm**2*mk*p**3 + fm**2*p**3 - mk*p**3)/mk)**2)/2 + 27*p*(-fm**2*mk*n*p - fm**2*mk*p**2 - fm**2*p**2 + 3*mk*p**2)/(2*mk) + 27*(-fm**4*n*p**2 - fm**4*p**3 + fm**2*mk*n*p**2 + fm**2*mk*p**3 + fm**2*p**3 - mk*p**3)/(2*mk))**(

In [117]:
fn_sol_0[1][fn].subs({mk: mk_, fm: fm_, p: p_, n:n_})

30.0000000000001

In [118]:
tn1_sol_0.subs({mk: mk_, fm: fm_, p: p_, n:n_, fn: 30})

240.000000000000

In [33]:
# tn1 case 1

In [34]:
tn1_sol_1 = tn1[1][tn]

In [35]:
eq30 = tn0[0][tn] - tn1[1][tn]

In [36]:
eq30

fm**2*p/4 + n - (-2*fn*mk + mk*n + mk*p - p + sqrt(4*fn*mk*n + 4*fn*mk*p + mk**2*n**2 + 2*mk**2*n*p + mk**2*p**2 - 2*mk*n*p - 2*mk*p**2 + p**2))/(2*mk) - (fm**2*p + 2*fn - 2*p)**2/(4*fm**2*p)

In [37]:
# supposing that mk > 0 and fm > 0

In [38]:
eq30 = sympy.simplify(eq30 * (2*mk*4*fm**2*p))

In [39]:
print(eq30)

4*fm**2*mk*n*p + 4*fm**2*mk*p**2 + 4*fm**2*p**2 - 4*fm**2*p*sqrt(4*fn*mk*n + 4*fn*mk*p + mk**2*n**2 + 2*mk**2*n*p + mk**2*p**2 - 2*mk*n*p - 2*mk*p**2 + p**2) - 8*fn**2*mk + 16*fn*mk*p - 8*mk*p**2


In [40]:
expr = 4*fm**2*p*sqrt(4*fn*mk*n + 4*fn*mk*p + mk**2*n**2 + 2*mk**2*n*p + mk**2*p**2 - 2*mk*n*p - 2*mk*p**2 + p**2)

In [41]:
eq30_mod = sympy.simplify(sympy.Eq((eq30 + expr)**2, (expr)**2))

In [42]:
eq30_mod = eq30_mod.lhs - eq30_mod.rhs

In [43]:
eq30_mod

-16*fm**4*p**2*(4*fn*mk*n + 4*fn*mk*p + mk**2*n**2 + 2*mk**2*n*p + mk**2*p**2 - 2*mk*n*p - 2*mk*p**2 + p**2) + 16*(fm**2*mk*n*p + fm**2*mk*p**2 + fm**2*p**2 - 2*fn**2*mk + 4*fn*mk*p - 2*mk*p**2)**2

In [44]:
fn_sol_0 = sympy.solve(eq30_mod, fn, dict=True)

In [45]:
fn_sol_0

[{fn: p},
 {fn: p - (9*p**2 - 3*(-fm**2*mk*n*p - fm**2*mk*p**2 - fm**2*p**2 + 3*mk*p**2)/mk)/(3*(-27*p**3 + sqrt(-4*(9*p**2 - 3*(-fm**2*mk*n*p - fm**2*mk*p**2 - fm**2*p**2 + 3*mk*p**2)/mk)**3 + (-54*p**3 + 27*p*(-fm**2*mk*n*p - fm**2*mk*p**2 - fm**2*p**2 + 3*mk*p**2)/mk + 27*(-fm**4*n*p**2 - fm**4*p**3 + fm**2*mk*n*p**2 + fm**2*mk*p**3 + fm**2*p**3 - mk*p**3)/mk)**2)/2 + 27*p*(-fm**2*mk*n*p - fm**2*mk*p**2 - fm**2*p**2 + 3*mk*p**2)/(2*mk) + 27*(-fm**4*n*p**2 - fm**4*p**3 + fm**2*mk*n*p**2 + fm**2*mk*p**3 + fm**2*p**3 - mk*p**3)/(2*mk))**(1/3)) - (-27*p**3 + sqrt(-4*(9*p**2 - 3*(-fm**2*mk*n*p - fm**2*mk*p**2 - fm**2*p**2 + 3*mk*p**2)/mk)**3 + (-54*p**3 + 27*p*(-fm**2*mk*n*p - fm**2*mk*p**2 - fm**2*p**2 + 3*mk*p**2)/mk + 27*(-fm**4*n*p**2 - fm**4*p**3 + fm**2*mk*n*p**2 + fm**2*mk*p**3 + fm**2*p**3 - mk*p**3)/mk)**2)/2 + 27*p*(-fm**2*mk*n*p - fm**2*mk*p**2 - fm**2*p**2 + 3*mk*p**2)/(2*mk) + 27*(-fm**4*n*p**2 - fm**4*p**3 + fm**2*mk*n*p**2 + fm**2*mk*p**3 + fm**2*p**3 - mk*p**3)/(2*mk))**(

In [46]:
fn_sol_0[1][fn].subs({mk: mk_, fm: fm_, p: p_, n:n_})

30.0000000000001

In [47]:
tn1_sol_1.subs({mk: mk_, fm: fm_, p: p_, n:n_, fn: 30})

29.9999999999999

In [48]:
# the right solutions
print(fn_sol_0[1][fn])
print(tn1_sol_1)

p - (9*p**2 - 3*(-fm**2*mk*n*p - fm**2*mk*p**2 - fm**2*p**2 + 3*mk*p**2)/mk)/(3*(-27*p**3 + sqrt(-4*(9*p**2 - 3*(-fm**2*mk*n*p - fm**2*mk*p**2 - fm**2*p**2 + 3*mk*p**2)/mk)**3 + (-54*p**3 + 27*p*(-fm**2*mk*n*p - fm**2*mk*p**2 - fm**2*p**2 + 3*mk*p**2)/mk + 27*(-fm**4*n*p**2 - fm**4*p**3 + fm**2*mk*n*p**2 + fm**2*mk*p**3 + fm**2*p**3 - mk*p**3)/mk)**2)/2 + 27*p*(-fm**2*mk*n*p - fm**2*mk*p**2 - fm**2*p**2 + 3*mk*p**2)/(2*mk) + 27*(-fm**4*n*p**2 - fm**4*p**3 + fm**2*mk*n*p**2 + fm**2*mk*p**3 + fm**2*p**3 - mk*p**3)/(2*mk))**(1/3)) - (-27*p**3 + sqrt(-4*(9*p**2 - 3*(-fm**2*mk*n*p - fm**2*mk*p**2 - fm**2*p**2 + 3*mk*p**2)/mk)**3 + (-54*p**3 + 27*p*(-fm**2*mk*n*p - fm**2*mk*p**2 - fm**2*p**2 + 3*mk*p**2)/mk + 27*(-fm**4*n*p**2 - fm**4*p**3 + fm**2*mk*n*p**2 + fm**2*mk*p**3 + fm**2*p**3 - mk*p**3)/mk)**2)/2 + 27*p*(-fm**2*mk*n*p - fm**2*mk*p**2 - fm**2*p**2 + 3*mk*p**2)/(2*mk) + 27*(-fm**4*n*p**2 - fm**4*p**3 + fm**2*mk*n*p**2 + fm**2*mk*p**3 + fm**2*p**3 - mk*p**3)/(2*mk))**(1/3)/3
(-2*fn*mk

In [49]:
fn_sol_a = fn_sol_0[1][fn]
tn_sol_a = tn1_sol_1

In [50]:
# the other way

In [51]:
eq20 = tp0[1][tp] - tp2[0][tp]
eq21 = tp1[0][tp] - tp2[0][tp]

In [52]:
tn0 = sympy.solve(eq20, tn, dict=True)
tn1 = sympy.solve(eq21, tn, dict=True)

In [53]:
tn0

[{tn: fm**2*p/4 + n - (fm**2*p + 2*fn - 2*p)**2/(4*fm**2*p)}]

In [54]:
tn1

[{tn: (-2*fn*mk + mk*n + mk*p - p - sqrt(4*fn*mk*n + 4*fn*mk*p + mk**2*n**2 + 2*mk**2*n*p + mk**2*p**2 - 2*mk*n*p - 2*mk*p**2 + p**2))/(2*mk)},
 {tn: (-2*fn*mk + mk*n + mk*p - p + sqrt(4*fn*mk*n + 4*fn*mk*p + mk**2*n**2 + 2*mk**2*n*p + mk**2*p**2 - 2*mk*n*p - 2*mk*p**2 + p**2))/(2*mk)}]

In [55]:
# tn1 case 0

In [56]:
tn1_sol_0 = tn1[0][tn]

In [57]:
eq30 = tn0[0][tn] - tn1[0][tn]

In [58]:
eq30

fm**2*p/4 + n - (-2*fn*mk + mk*n + mk*p - p - sqrt(4*fn*mk*n + 4*fn*mk*p + mk**2*n**2 + 2*mk**2*n*p + mk**2*p**2 - 2*mk*n*p - 2*mk*p**2 + p**2))/(2*mk) - (fm**2*p + 2*fn - 2*p)**2/(4*fm**2*p)

In [59]:
# supposing that mk > 0 and fm > 0

In [60]:
eq30 = sympy.simplify(eq30 * (2*mk*4*fm**2*p))

In [61]:
print(eq30)

4*fm**2*mk*n*p + 4*fm**2*mk*p**2 + 4*fm**2*p**2 + 4*fm**2*p*sqrt(4*fn*mk*n + 4*fn*mk*p + mk**2*n**2 + 2*mk**2*n*p + mk**2*p**2 - 2*mk*n*p - 2*mk*p**2 + p**2) - 8*fn**2*mk + 16*fn*mk*p - 8*mk*p**2


In [62]:
expr = 4*fm**2*p*sqrt(4*fn*mk*n + 4*fn*mk*p + mk**2*n**2 + 2*mk**2*n*p + mk**2*p**2 - 2*mk*n*p - 2*mk*p**2 + p**2)

In [63]:
eq30_mod = sympy.simplify(sympy.Eq((eq30 - expr)**2, (-expr)**2))

In [64]:
eq30_mod = eq30_mod.lhs - eq30_mod.rhs

In [65]:
eq30_mod

-16*fm**4*p**2*(4*fn*mk*n + 4*fn*mk*p + mk**2*n**2 + 2*mk**2*n*p + mk**2*p**2 - 2*mk*n*p - 2*mk*p**2 + p**2) + 16*(fm**2*mk*n*p + fm**2*mk*p**2 + fm**2*p**2 - 2*fn**2*mk + 4*fn*mk*p - 2*mk*p**2)**2

In [66]:
fn_sol_0 = sympy.solve(eq30_mod, fn, dict=True)

In [67]:
fn_sol_0[1][fn].subs({mk: mk_, fm: fm_, p: p_, n:n_})

30.0000000000001

In [68]:
tn1_sol_0.subs({mk: mk_, fm: fm_, p: p_, n:n_, fn: 30})

240.000000000000

In [69]:
# tn1 case 1

In [70]:
tn1_sol_0 = tn1[1][tn]

In [71]:
eq30 = tn0[0][tn] - tn1[0][tn]

In [72]:
eq30

fm**2*p/4 + n - (-2*fn*mk + mk*n + mk*p - p - sqrt(4*fn*mk*n + 4*fn*mk*p + mk**2*n**2 + 2*mk**2*n*p + mk**2*p**2 - 2*mk*n*p - 2*mk*p**2 + p**2))/(2*mk) - (fm**2*p + 2*fn - 2*p)**2/(4*fm**2*p)

In [73]:
# supposing that mk > 0 and fm > 0

In [74]:
eq30 = sympy.simplify(eq30 * (2*mk*4*fm**2*p))

In [75]:
print(eq30)

4*fm**2*mk*n*p + 4*fm**2*mk*p**2 + 4*fm**2*p**2 + 4*fm**2*p*sqrt(4*fn*mk*n + 4*fn*mk*p + mk**2*n**2 + 2*mk**2*n*p + mk**2*p**2 - 2*mk*n*p - 2*mk*p**2 + p**2) - 8*fn**2*mk + 16*fn*mk*p - 8*mk*p**2


In [76]:
expr = 4*fm**2*p*sqrt(4*fn*mk*n + 4*fn*mk*p + mk**2*n**2 + 2*mk**2*n*p + mk**2*p**2 - 2*mk*n*p - 2*mk*p**2 + p**2)

In [77]:
eq30_mod = sympy.simplify(sympy.Eq((eq30 - expr)**2, (-expr)**2))

In [78]:
eq30_mod = eq30_mod.lhs - eq30_mod.rhs

In [79]:
eq30_mod

-16*fm**4*p**2*(4*fn*mk*n + 4*fn*mk*p + mk**2*n**2 + 2*mk**2*n*p + mk**2*p**2 - 2*mk*n*p - 2*mk*p**2 + p**2) + 16*(fm**2*mk*n*p + fm**2*mk*p**2 + fm**2*p**2 - 2*fn**2*mk + 4*fn*mk*p - 2*mk*p**2)**2

In [80]:
fn_sol_0 = sympy.solve(eq30_mod, fn, dict=True)

In [81]:
fn_sol_0[1][fn].subs({mk: mk_, fm: fm_, p: p_, n:n_})

30.0000000000001

In [82]:
tn1_sol_0.subs({mk: mk_, fm: fm_, p: p_, n:n_, fn: 30})

29.9999999999999

In [83]:
# the other true solution
print(fn_sol_0[1][fn])
print(tn1_sol_0)

p - (9*p**2 - 3*(-fm**2*mk*n*p - fm**2*mk*p**2 - fm**2*p**2 + 3*mk*p**2)/mk)/(3*(-27*p**3 + sqrt(-4*(9*p**2 - 3*(-fm**2*mk*n*p - fm**2*mk*p**2 - fm**2*p**2 + 3*mk*p**2)/mk)**3 + (-54*p**3 + 27*p*(-fm**2*mk*n*p - fm**2*mk*p**2 - fm**2*p**2 + 3*mk*p**2)/mk + 27*(-fm**4*n*p**2 - fm**4*p**3 + fm**2*mk*n*p**2 + fm**2*mk*p**3 + fm**2*p**3 - mk*p**3)/mk)**2)/2 + 27*p*(-fm**2*mk*n*p - fm**2*mk*p**2 - fm**2*p**2 + 3*mk*p**2)/(2*mk) + 27*(-fm**4*n*p**2 - fm**4*p**3 + fm**2*mk*n*p**2 + fm**2*mk*p**3 + fm**2*p**3 - mk*p**3)/(2*mk))**(1/3)) - (-27*p**3 + sqrt(-4*(9*p**2 - 3*(-fm**2*mk*n*p - fm**2*mk*p**2 - fm**2*p**2 + 3*mk*p**2)/mk)**3 + (-54*p**3 + 27*p*(-fm**2*mk*n*p - fm**2*mk*p**2 - fm**2*p**2 + 3*mk*p**2)/mk + 27*(-fm**4*n*p**2 - fm**4*p**3 + fm**2*mk*n*p**2 + fm**2*mk*p**3 + fm**2*p**3 - mk*p**3)/mk)**2)/2 + 27*p*(-fm**2*mk*n*p - fm**2*mk*p**2 - fm**2*p**2 + 3*mk*p**2)/(2*mk) + 27*(-fm**4*n*p**2 - fm**4*p**3 + fm**2*mk*n*p**2 + fm**2*mk*p**3 + fm**2*p**3 - mk*p**3)/(2*mk))**(1/3)/3
(-2*fn*mk

In [84]:
fn_sol_b = fn_sol_0[1][fn]
tn_sol_b = tn1_sol_0

In [85]:
fn_sol_a == fn_sol_b

True

In [86]:
tn_sol_a == tn_sol_b

True